# Исследование надежности заемщиков


## Описание проекта
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## План выполнения проекта
1. Откроем и изучим данные
2. Проведем предобработку данных
    * проверка на наличие пропусков/дубликатов
    * заполнение пропусков
    * удаление аномалий
    * преобразование типов данных
    * обработка дубликатов
    * создание дополнительных категориальных признаков

# Загрузка и обзор данных

In [1]:
# импорт библиотеки pandas
import pandas as pd

In [2]:
# загрузка файла
try:
    data = pd.read_csv('data.csv')
except:
    data = pd.read_csv('/another_folder/datasets/data.csv')

Выведем первые 20 строчек датафрейма data на экран.

In [3]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выведем основную информацию о датафрейме с помощью метода **info()**

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


# Предобработка данных

## Удаление пропусков

Выведем количество пропущенных значений для каждого столбца.

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Таким образом, в столбце `days_employed` и `total_income ` имеются пропущенные значения.

Начнем со столбца с пропущенными значениями `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце логичнее медианным значением по каждому типу из столбца `income_type`. 

Например, у человека с типом занятости сотрудник пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [6]:
# перебираем каждый тип занятости в наборе уникальных значений столба income_type
for t in data['income_type'].unique():
    # на каждом шаге цикла с помощью атрибута loc выбираем строки,
    # в которых в income_type текущий тип занятости (t) и есть пропуски в total_income 
    # и записываем в них медианное значение total_income среди строк с текущим типом занятости (t)
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

In [7]:
# проверим, что все пропуски заполнены
print('Количество пропусков после заполнения:', data['total_income'].isna().sum())

Количество пропусков после заполнения: 0


## Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. 

Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Обработаем значения в этом столбце путем замены всех отрицательные значения положительными с помощью метода **abs()**.

In [8]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [9]:
data.groupby('income_type')['days_employed'].median()

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть.

Выведем перечень уникальных значений столбца `children`.

In [10]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

Да уж, из крайности в крайность... толи childfree и их ярые противники закрались, но скорее это всего лишь аномальные значения. Удалим строки с этими зачениями

In [11]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.

In [12]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

Теперь все отлично!

Вернемся к вопросу заполнения пропусков в столбце `days_employed`. Заполним пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`.

Выбор медианы объясняется тем, что медианное значение не подвергается воздействию выбросов (аномалий) в данных, как среднее значение. А заполнение нулями - сильно повлияет на статистику.

In [13]:
# для каждого уникального значения занятости income_type перебираем таблицу
for t in data['income_type'].unique():

# каждой строке с пропуском в поле days_employed и типом занятости t 
# присваиваем медианное значение по типу занятости t
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены.

In [14]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Пропуски заполнены!

Проведем изменение типа данных столбца `total_income` с вещественного на целочисленный с помощью метода **astype()**

In [15]:
data['total_income'] = data['total_income'].astype('int')

## Удаление дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.

In [16]:
data['education'] = data['education'].str.lower()

Выведем на экран количество строк-дубликатов в данных, удалим их.

In [17]:
# выведем число дубликатов
data.duplicated().sum()

71

In [18]:
data = data.drop_duplicates()

## Категоризация данных

Cоздадим столбец `total_income_category` с категориями:
* 0–30000 — `E`;
* 30001–50000 — `D`;
* 50001–200000 — `C`;
* 200001–1000000 — `B`;
* 1000001 и выше — `A`.

Для этого создадим функцию с именем `categorize_income()` и используем метод **apply()**.

In [19]:
# создаем функцию categorize_income()
def categorize_income(income):
    if income <= 30000:
        return 'E'
    if income <= 50000:
        return 'D'
    if income <= 200000:
        return 'C'
    if income <= 1000000:
        return 'B'
    return 'A'

In [20]:
# примените функцию методом apply()
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [21]:
# выведем 5 строк 
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C


Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.

In [22]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Сформируем новый столбец `purpose_category`, в который войдут следующие категории:

* `операции с автомобилем`,
* `операции с недвижимостью`,
* `проведение свадьбы`,
* `получение образования`.

In [23]:
# создайте функцию categorize_purpose()
def categorize_purpose(purpose):
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'жиль' in purpose:
        return 'операции с недвижимостью'
    if 'недвиж' in purpose:
        return 'операции с недвижимостью'
    if 'свадь' in purpose:
        return 'проведение свадьбы'
    if 'образов' in purpose:
        return 'получение образования'        
    return 'неотложные нужды'

In [24]:
# примените функцию методом apply()
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

Таким образом, нами были выполнены следующие шаги:
  * проверены данные на наличие пропусков
  * заполнены пропуски
  * удалены аномалии
  * выполнено преобразование типов данных
  * удалены полные дубликаты
  * проведена категоризация данных
  
Можно приступать к исследованию зависимостей.

# Иcследование данных

## Зависимость между количеством детей и возвратом кредита в срок

Создадим сводную таблицу с группировкой по количеству детей в семье.
В качестве агрегатных значений в каждой группе вычислим 
* количество записей `count` 
* количество задолженностей `sum`

In [25]:
# для дальнейшего удобства работы с таблицей осуществим сброс индексов 
# и уйдем от двойной индексации
debt_children = data.pivot_table(index = 'children', 
                                 values = 'debt', 
                                 aggfunc = {'count', 'sum', 'mean'})\
                    .reset_index()\
                    .rename_axis(None, axis=1)
debt_children

,children,count,mean,sum
0,0,14091,0.075438,1063
1,1,4808,0.092346,444
2,2,2052,0.094542,194
3,3,330,0.081818,27
4,4,41,0.097561,4
5,5,9,0.000000,0


In [26]:
# посчитаем процент каждой группы заемщиков в выборке для оценки репрезентативности результатов
debt_children ['percent_group'] = round((debt_children['count'] / len(data) * 100), 2)

# преобразуем долю в проценты для удобства
debt_children ['mean'] = round((debt_children ['mean'] * 100), 2)

#Для удобства переименуем название столбцов
debt_children.rename(columns={'children': 'Кол-во детей', 
                              'count' : 'Кол-во заемщиков', 
                              'sum' : 'Кол-во просрочек', 
                              'mean' : '% просрочек', 
                              'percent_group' : '% группы'})


,Кол-во детей,Кол-во заемщиков,% просрочек,Кол-во просрочек,% группы
0,0,14091,7.54,1063,66.06
1,1,4808,9.23,444,22.54
2,2,2052,9.45,194,9.62
3,3,330,8.18,27,1.55
4,4,41,9.76,4,0.19
5,5,9,0.00,0,0.04



**Вывод:** 
* Семьи с 3 и более детьми становятся заемщиками банка очень редко - процент таких заемщиков составляет в общей сложности 1,78% в кредитной выборке. Поэтому эти данные не будем использовать для поиска закономерностей.
* **Лидерами по возврату кредитов** являются семьи без детей, процент задолженности составляет 7.54%. Процент таких заемщиков в выборке банка 66%, таким образом можно сделать вывод, что люди без детей, охотнее привлекают заемные средства.
* Семьи с детьми возвращают кредиты хуже. С увеличением количества детей ухудшается и качество кредитов и уменьшается их доля в кредитном портфеле банка.

## Зависимость между семейным положением и возвратом кредита в срок

Создадим сводную таблицу с группировкой по семейному положению.
В качестве агрегатных значений в каждой группе вычислим 
* количество записей `count` 
* количество задолженностей `sum`

In [27]:
# для дальнейшего удобства работы с таблицей осуществим сброс индексов 
# и уйдем от двойной индексации
debt_family = data.pivot_table(index = 'family_status_id', 
                               values = 'debt', 
                               aggfunc = {'count', 'sum', 'mean'})\
                   .reset_index()\
                   .rename_axis(None, axis=1)
debt_family

,family_status_id,count,mean,sum
0,0,12261,0.075606,927
1,1,4134,0.093130,385
2,2,951,0.066246,63
3,3,1189,0.070648,84
4,4,2796,0.097639,273


Cоздадим новую таблицу `family_dict` 

In [28]:
# создадим новую таблицу family_dict, удалим дубликаты и сбросим индексы
family_dict = data[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True)
family_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [29]:
# из таблицы family_dict подставим family_status, 
# соответствующие family_status_id путем объединения таблиц
debt_family = debt_family.merge(family_dict, on='family_status_id', how='left')

In [30]:
# преобразуем долю в проценты для удобства
debt_family ['mean'] = round((debt_family ['mean'] * 100), 2)

In [31]:
# посчитаем долю каждой группы заемщиков в выборке для оценки репрезентативности результатов
debt_family ['percent_group'] = round((debt_family['count'] / len(data) * 100), 2)

In [32]:
#Для удобства переименуем название столбцов
debt_family[['family_status', 'count' ,'sum', 'mean', 'percent_group']].rename(columns={'family_status': 'Семейное положение', 
                            'count' : 'Кол-во заемщиков',
                            'sum' : 'Кол-во просрочек',
                            'mean' : '% просрочек', 
                            'percent_group' : '% группы'})

,Семейное положение,Кол-во заемщиков,Кол-во просрочек,% просрочек,% группы
0,женат / замужем,12261,927,7.56,57.48
1,гражданский брак,4134,385,9.31,19.38
2,вдовец / вдова,951,63,6.62,4.46
3,в разводе,1189,84,7.06,5.57
4,Не женат / не замужем,2796,273,9.76,13.11


**Вывод:** 
* самыми надежными заемщиками в исследуемой выборке являются вдовы/вдовцы и лица, находящиеся в разводе. При этом доля таких заемщиков составляет 4,5% и 5,6% соответсвенно.
* Самыми проблемными заемщиками являются не женатые/ не замужние граждане.

## Зависимость между уровнем дохода и возвратом кредита в срок

Создадим сводную таблицу с группировкой по категорийному признаку `total_income_category`. В качестве агрегатных значение в каждой группе вычислим 
* количество записей `count` 
* количество задолженностей `sum`

In [33]:
# для дальнейшего удобства работы с таблицей осуществим сброс индексов и уйдем от двойной индексации
debt_income = data.pivot_table(index = 'total_income_category', 
                               values = 'debt', 
                               aggfunc = {'count', 'sum', 'mean'})\
                   .reset_index()\
                   .rename_axis(None, axis=1)
debt_income

,total_income_category,count,mean,sum
0,A,25,0.080000,2
1,B,5014,0.070602,354
2,C,15921,0.084982,1353
3,D,349,0.060172,21
4,E,22,0.090909,2


In [34]:
# преобразуем долю в проценты для удобства
debt_income ['mean'] = round((debt_income ['mean'] * 100), 2)

In [35]:
# посчитаем долю каждой группы заемщиков в выборке для оценки репрезентативности результатов
debt_income ['percent_group'] = round((debt_income['count'] / len(data) * 100), 2)

In [36]:
# сделаем столбец с расшифровкой категории дохода заемщика
for t_i_c in debt_income['total_income_category']:
    try:
        if t_i_c == 'E':
            debt_income.loc[debt_income['total_income_category'] == t_i_c, 'level_income'] = 'доход до 30000'
        elif t_i_c == 'D':
            debt_income.loc[debt_income['total_income_category'] == t_i_c, 'level_income'] = 'доход от 30001 до 50000'
        elif t_i_c == 'C':
            debt_income.loc[debt_income['total_income_category'] == t_i_c, 'level_income'] = 'доход от 50001 до 200000'
        elif t_i_c == 'B':
            debt_income.loc[debt_income['total_income_category'] == t_i_c, 'level_income'] = 'доход от 200001 до 1000000'
        elif t_i_c == 'A':
            debt_income.loc[debt_income['total_income_category'] == t_i_c, 'level_income'] = 'доход более 1000001'
    except:
        pass

In [37]:
#Для удобства переименуем название столбцов
debt_income[['total_income_category', 'level_income', 'count','sum', 'mean', 'percent_group']]\
.rename(columns={'total_income_category': 'Категория дохода', 
                 'level_income': 'Уровень дохода', 
                 'count' : 'Кол-во заемщиков',
                 'sum' : 'Кол-во просрочек', 
                 'mean' : '% просрочек', 
                 'percent_group' : '% группы'})

,Категория дохода,Уровень дохода,Кол-во заемщиков,Кол-во просрочек,% просрочек,% группы
0,A,доход более 1000001,25,2,8.00,0.12
1,B,доход от 200001 до 1000000,5014,354,7.06,23.51
2,C,доход от 50001 до 200000,15921,1353,8.50,74.64
3,D,доход от 30001 до 50000,349,21,6.02,1.64
4,E,доход до 30000,22,2,9.09,0.10


**Вывод:** 
* Минимальный уровень просрочки наблюдается у заемщиков категории **D** (уровень дохода от 30001 до 50000).
* Максимальный уровень просроченной задолженности у лиц категории **E** с доходами до 30000. 
* Категории **D**, **E** в исследуемой выборке являются несущественными.
* Основной сегмент заемщиков, представлен клиентами с доходами от 50001 до 1000000 (**категории C и B**). 
* Более надежными заемщиками являются клиенты категории **B**. 

## Зависимость возврата в срок от цели кредита

Создадим сводную таблицу с группировкой по категорийному признаку `purpose_category`. В качестве агрегатных значение в каждой группе вычислим 
* количество записей `count` 
* количество задолженностей `sum`

In [38]:
# для дальнейшего удобства работы с таблицей осуществим сброс индексов и уйдем от двойной индексации
debt_purpose = data.pivot_table(index = 'purpose_category', 
                                values = 'debt', 
                                aggfunc = {'count', 'sum'})\
                   .reset_index()\
                   .rename_axis(None, axis=1)
debt_purpose

,purpose_category,count,sum
0,операции с автомобилем,4279,400
1,операции с недвижимостью,10751,780
2,получение образования,3988,369
3,проведение свадьбы,2313,183


In [39]:
# расчитаем процент невозврата в каждой группе 
debt_purpose ['percent_debt'] = round((debt_purpose['sum'] / debt_purpose['count'] * 100), 2)
# посчитаем долю каждой группы заемщиков в выборке для оценки репрезентативности результатов
debt_purpose ['percent_group'] = round((debt_purpose['count'] / len(data) * 100), 2)

In [40]:
#Для удобства переименуем название столбцов
debt_purpose[['purpose_category', 'count','sum', 'percent_debt', 'percent_group']]\
.rename(columns={'purpose_category': 'Цель кредита', 
                 'count' : 'Кол-во заемщиков', 
                 'sum' : 'Кол-во просрочек',
                 'percent_debt' : '% просрочек', 
                 'percent_group' : '% группы кредитов'})

,Цель кредита,Кол-во заемщиков,Кол-во просрочек,% просрочек,% группы кредитов
0,операции с автомобилем,4279,400,9.35,20.06
1,операции с недвижимостью,10751,780,7.26,50.40
2,получение образования,3988,369,9.25,18.70
3,проведение свадьбы,2313,183,7.91,10.84


**Вывод:** 
* Основная доля кредитов привлекалась (более 50%) для операций с недвижимость, в данной категории минимальный уровень просрочки.
* Кредиты на проведения свадьбы находятся на втором месте по благонадержности. Процент таких кредитов составляет 11% в исследуемой выборке.
* Кредитов, направленных на образование, почти 4000, при этом уровень просрочки составляет 9.25%. 
* Немного больше кредитов, привлечено на операции с автомобилями, но и качество данных кредитов также немного хуже.
* Лидерами по просроченной задолженности в данной выборке являются кредиты с автомобилями.

## Анализ причины появления пропусков в исходных данных

Проведем маленькое расследование.

Рассмотрим исходный датафрей

In [41]:
try:
    data_nan= pd.read_csv('data.csv')
except:
    data_nan = pd.read_csv('/another_folder/datasets/data.csv')

In [42]:
# выведем информацию о выборке, в которой пропущены данные о доходах заемщика
data_nan[data_nan['total_income'].isna()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2174 non-null   int64  
 1   days_employed     0 non-null      float64
 2   dob_years         2174 non-null   int64  
 3   education         2174 non-null   object 
 4   education_id      2174 non-null   int64  
 5   family_status     2174 non-null   object 
 6   family_status_id  2174 non-null   int64  
 7   gender            2174 non-null   object 
 8   income_type       2174 non-null   object 
 9   debt              2174 non-null   int64  
 10  total_income      0 non-null      float64
 11  purpose           2174 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


In [43]:
# выберем записи с отсуствующими значениями
data_nan = data_nan[data_nan['total_income'].isna()]

Проверим предположение о том, что пропуск значение в поле `total_income` и `days_employed` коррелируют друг с другом.

Cгруппируем по столбцу тип занятости (`income_type`) и оценим интересующую нас колонку `days_employed`. Если есть хотя бы одно не нулевое значение, мы получим сумму, отличную от нуля

In [44]:
data_nan.groupby('income_type')['days_employed'].sum()

income_type
госслужащий        0.0
компаньон          0.0
пенсионер          0.0
предприниматель    0.0
сотрудник          0.0
Name: days_employed, dtype: float64

In [45]:
# проверим как распределены пропуски по группам занятости
data_nan.groupby('income_type')['children'].count()

income_type
госслужащий         147
компаньон           508
пенсионер           413
предприниматель       1
сотрудник          1105
Name: children, dtype: int64

**Вывод**: 
* массовые пропуски в данных согласно проведеному анализу не связаны с человеческим фактором и скорее всего являются результатом технического сбоя (например, не подгрузились данные из одной системы учета сотрудников в общую базу по кредитам).
* Единичные пропуски скорее всего связаны с человеческим фактором. 

# Вывод

В ходе проекта была исследована платёжеспособность клиентов банка по представленным кредитным отделом данным.

Целью являлось проверка гипотезы влияния семейного положения и количества детей клиента на факты погашения кредита в срок. Результаты проверки этих гипотез должны быть учтены при построении модели кредитного скоринга.

**Перед проверкой гипотез был проведен обзор и предобработка данных:**

* Проверка данных на пропущенные значения выявила пропуски в двух столбцах `income_type` и `days_employed`. Анализ причин появления пропусков в данных столбцах указывает, что данные в данных колонках пропущены в результате технического сбоя.
Пропуски в столбце `income_type`, который оказывает существенное влияние на анализ выборки, были заполнены медианными значениями для каждого типа занятости.
Пропуски в `days_employed` на текущем этапе исследования не заполнялись, так как в исследовании вышеуказанных гипотез не участвовали.

* Были удалены артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке и полные дубликаты.

* Проведена категоризация данных по столбцам 
   `total_income` - все клиенты были разделены на категории в зависимости от размера их дохода.
   `purpose` - все цели привлечения денежных средств клиентами были разделены на 4 группы.

**Проверены гипотезы и выявлены следующие закономерности:**

*Влияние семейного положения на возврат кредита в срок*

Гипотеза, о том, что семейное положение оказывает влияние на возврат крдеита в срок подтвердилась.
* Люди, находящиеся, либо ранее бывшие в официальных отношениях, возвращают кредиты лучше, чем, граждане не состоящие в браке.
* Самыми надежными заемщиками в исследуемой выборке являются вдовы/вдовцы и лица, находящиеся в разводе. При этом совокупный портфель кредитов таких заемщиков около 10%. 6,6% и 7% составляют кредиты, возвращенные не в срок.
* Самыми проблемными заемщиками являются не женатые/не замужние граждане (9,8% просроченных кредитов, 13% в кредитном портфеле банка) и граждане, состоящие в гражданском браке (9,3% просроченных кредитов, 19% в кредитном портфеле банка).
* Максимальный количество кредитов выдано гражданам, находящимся в официальном браке - 57,5% в кредитном портфеле банка. Процент невозврата 7,6%.

Таким образом, заемщики, находящиеся в официальном браке, либо ранее в нем состоящие, являются более ответственными в плане платежной дисциплины.

*Влияние количества детей на возврат кредита в срок*


Гипотеза о влиянии наличия детей у клиента на возврат кредита потверждена. 
* При наличии детей вероятность возврата кредита в срок снижается.
* Семьи с 3 и более детьми становятся заемщиками банка очень редко - процент таких заемщиков составляет в общей сложности 1,78% в кредитной выборке. Поэтому эти данные не используем для оценки наличия закономерностей.
* Лидерами по возврату кредитов являются семьи без детей, процент задолженности составляет 7.54%. Процент таких заемщиков в выборке банка 66%, таким образом можно сделать вывод, что люди без детей, охотнее привлекают и возвращают в срок заемные средства.
* Семьи с детьми возвращают кредиты хуже. С увеличением количества детей ухудшается и качество кредитов и уменьшается их доля в кредитном портфеле банка:
     - клиенты с одним ребенком составляют 22,5% от общего количества заемщиков, 9,2% кредитов возвращены не в срок.
     - клиенты, имеющие 2 детей -           9,6% от общего количества заемщиков, 9,5% кредитов возвращены не в срок.

*Влияние уровня дохода на возврат кредита в срок*


В результате выявления зависимостей между уровнем дохода и возратом кредита установлено:
* Общий кредитный портфель клиентов категории D (уровень дохода от 30001 до 50000), E (доходы до 30000) и A (доходы свяше 1000000) составляет менее 2%, и является несущественным при проведении анализа.
* Основной сегмент заемщиков, представлен клиентами с доходами от 50001 до 1000000 (категории C и B).
* Более надежными заемщиками являются клиенты категории B (доходы свыше 200000), процент просрочек составляет 7%, а кредитный портфель - 23,5% от общего кредитного портфеля.
* Количество клиентов категории C (доходы до 200000) почти в 3 раза больше, а уровень просроченной задолженности на 1,5% выше.
* Этот результат мог оказаться иным, если бы не пропуски в данных, которые были заполнены нами на основании медианных значений по типу занятости.

*Влияние цели кредита на его возврат в срок*
* Основная доля кредитов привлекалась (более 50%) для операций с недвижимость, в данной категории минимальный уровень просрочки - 7,3%, что объясняется скорее всего тем, что в качестве обеспечения по кредиту выступало недвижимое имущество.
* Кредиты на проведения свадьбы находятся на втором месте по благонадержности. Процент таких кредитов составляет 11% в исследуемой выборке, и 7,9% - невозврата в установленный срок.
* Кредитов, направленных на образование, почти 4000 или 18,7% от кредитного портфеля, при этом уровень просрочки составляет 9.25%.
* Немного больше кредитов (20% из выборки), привлечено на операции с автомобилями, но и качество данных кредитов также немного хуже (9,35% просроченной задолженности).
* Лидерами по просроченной задолженности в данной выборке являются кредиты с автомобилями. 
* Таким образом кредиты на приобретени автомобилей являются более рискованными, чем беззалоговые крдеиты на проведение свадебных мероприятий.


На основании вышеизложенного **наиболее привлекательным** для банка клиентом является гражданин 
- состоящий/ранее бывший в браке;
- не имеющий несовершеннолетних детей;
- с уровнем дохода свыше 200000

Предпочтительная цель привлечения кредитных средств:
- приобретение недвижимости
- проведение свадебных мероприятий

При этом можно рекомендовать 
1. при построении скоринговой модели сегментировать на более мелкие категории уровень дохода заемщика;
2. проработать вопрос дополнения/заполнения пропусков данными из других систем банка.

Согласно проведенно анализа можно сделать следующие выводы:
 - клиенты, находящиеся в браке, либо в разводе обладают лучшей платежной дисциплиной по сравнению с людьми, не состоящими в официальных отношениях;
 - самыми надежными клиентами банка являются вдовы и вдовцы;
 - при наличии детей вероятность возврата кредита в срок снижается;
 - прослеживается тендеция, что заемщики с большим количеством детей реже берут кредиты.